In [4]:
import os
import numpy as np
from parflow.tools.io import read_pfb
import cv2
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.patches import Rectangle

In [12]:
input_dir = '<path/to/daily/parflow/output/files>'
image_dir = 'path/to/save/daily/static/images'

water_year = 2024
days_in_wy = 365 #note this is one less than real number of days due to 6-hour offset when processing CONUS2

# Define variable list using ParFlow/CLM variable names
var_list = ['WTD', 'SM', 'flow', 'swe_out', 'eflx_lh_tot']

### Step 1: Create static images from daily files

In [13]:
for var in var_list:
    print(f'Creating static plots for {var}')

    for i in range(1, days_in_wy+1):

        file = f'{input_dir}/{var}.{water_year}.daily.{i:0{3}}.pfb'
        outfile = (f'{image_dir}/{var}/{var}.{water_year}.daily.{i:0{3}}.png')

        pf_data = read_pfb(file).squeeze()

        plt.figure()

        # Variable-specfiic adjustments and plotting features
        if var == 'WTD':
            pf_data[pf_data < 0.01] = 0.01
            pf_data[pf_data == 292.00000000000006] = np.nan
            plt.imshow(pf_data, origin='lower', cmap='RdYlBu_r',norm=colors.LogNorm(vmin = 0.01, vmax = 300))
            bounds = [0.1, 1, 10, 100]

        elif var == 'swe_out':
            pf_data[pf_data == -9999.0] = np.nan
            plt.imshow(pf_data, origin='lower', cmap='Blues',vmin = 0.0, vmax = 1700)
            bounds = [0, 750, 1500]
    
        elif var == 'SM':
            pf_data[pf_data == 0.0] = np.nan
            plt.imshow(pf_data[9, :, :], origin='lower', cmap='plasma_r',vmin = 0.01, vmax = 0.5)
            bounds = [0.1, 0.3, 0.5]

        elif var == 'flow':
            pf_data = pf_data/3600
            pf_data[pf_data == 0.0] = np.nan
            plt.imshow(pf_data, origin='lower', cmap='plasma_r',norm=colors.LogNorm(vmin = 0.01, vmax = 100000.))
            bounds = [0.1, 100, 20000]
    
        elif var == 'eflx_lh_tot':
            pf_data[pf_data == -9999.0] = np.nan
            plt.imshow(pf_data, origin='lower', cmap='plasma',vmin = 0.0, vmax = 450)
            bounds = [0, 200, 400]


        plt.axis('off')
        cbar = plt.colorbar(orientation='horizontal', shrink=0.15, ticks = bounds,anchor=(0.65,2.8)) 
        cbar.ax.tick_params(labelsize=6,labelfontfamily='sans-serif')
    
        if var == 'WTD':
            cbar.ax.set_xticklabels(['0.1', '1.0', '10', '100'])
            cbar.set_label('WT depth (m)', fontsize=8, fontfamily="sans-serif")
        elif var == 'swe_out':
            cbar.ax.set_xticklabels(['0', '750', '>1500'])
            cbar.set_label('SWE (mm)', fontsize=8, fontfamily="sans-serif")
        elif var == 'SM':
            cbar.ax.set_xticklabels(['0.1', '0.3', '0.5'])
            cbar.set_label('Soil Moisture (-)', fontsize=8, fontfamily="sans-serif")
        elif var == 'flow':
            cbar.ax.set_xticklabels(['0.1', '100', '20,000'])
            cbar.set_label('Flow (CMS)', fontsize=8, fontfamily="sans-serif")
        elif var == 'eflx_lh_tot':
            cbar.ax.set_xticklabels(['0', '200', '>400'])
            cbar.set_label('LH (W/m2)', fontsize=8, fontfamily="sans-serif")

        # create a progress bar as a gray rectangle
        plt.text(200, 550, 'Oct Mar Sep',fontsize=10, fontfamily="sans-serif")
        width = 1100 * (i/days_in_wy)
        plt.gca().add_patch(Rectangle((200, 550),width,150,linewidth=0,edgecolor='none',facecolor='gray',alpha=0.7))
        
        plt.title(f'Water Year {water_year}')
        plt.savefig(outfile, dpi=500,bbox_inches='tight')
        plt.clf()
        plt.close()

Creating static plots for flow
Creating static plots for swe_out
Creating static plots for eflx_lh_tot


### Step 2: Create .mp4 video from static images

In [ ]:
for var in var_list:
    print(f'Creating video for {var}')
    video_name = f'{var}_{water_year}.mp4'

    var_image_dir = f'{image_dir}/{var}'
    
    # Get a list of images from the folder, sorted by name or order
    images = [img for img in os.listdir(var_image_dir) if img.endswith('.png')]
    images.sort()  # Sort the images if they are numbered or named sequentially

    # Read the first image to get the dimensions (width and height)
    first_image = cv2.imread(os.path.join(var_image_dir, images[0]))
    height, width, _ = first_image.shape

    # Codec for .mp4
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') 

    # Frames per second
    fps = 25  
    out = cv2.VideoWriter(video_name, fourcc, fps, (width, height))

    # Loop through all images and add them to the video
    for image in images:
        img = cv2.imread(os.path.join(var_image_dir, image))

        # Write the image to the video
        out.write(img) 

    # Release the video writer
    out.release()

    print(f"Video saved as {video_name}")